In [1]:
import json
import joblib
from tqdm import tqdm
from pathlib import Path
from timeit import default_timer as timer

import pandas as pd
import numpy as np
from io import BytesIO
from PIL import Image
import h5py

import catboost as cb
import lightgbm as lgb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from timm import create_model

from accelerate import Accelerator

from isic_helper import DotDict
from isic_helper import get_folds
from isic_helper import time_to_str

In [2]:
INPUT_PATH = Path("../input/isic-2024-challenge/")
CB_ARTIFACTS_INPUT_PATH = Path(f"../input/isic-scd-cb-train/")
LGB_V1_ARTIFACTS_INPUT_PATH = Path(f"../input/isic-scd-lgb-v1-train/")
LGB_V2_ARTIFACTS_INPUT_PATH = Path(f"../input/isic-scd-lgb-v2-train/")
RESNET18_ARTIFACTS_INPUT_PATH = Path(f"../input/isic-scd-resnet18-train/")

cb_cfg = DotDict()
cb_cfg.models_output_dir = "models"
cb_cfg.model_name = "cb_v1"

lgb_v1_cfg = DotDict()
lgb_v1_cfg.models_output_dir = "models"
lgb_v1_cfg.model_name = "lgb_v1"

lgb_v2_cfg = DotDict()
lgb_v2_cfg.models_output_dir = "models"
lgb_v2_cfg.model_name = "lgb_v2"

with open(CB_ARTIFACTS_INPUT_PATH / "run_metadata.json", "r") as f:
    cb_run_metadata = json.load(f)
    
with open(LGB_V1_ARTIFACTS_INPUT_PATH / f"{lgb_v1_cfg.model_name}_run_metadata.json", "r") as f:
    lgb_v1_run_metadata = json.load(f)

with open(LGB_V2_ARTIFACTS_INPUT_PATH / f"{lgb_v2_cfg.model_name}_run_metadata.json", "r") as f:
    lgb_v2_run_metadata = json.load(f)

with open(RESNET18_ARTIFACTS_INPUT_PATH / "run_metadata.json", "r") as f:
    resnet18_run_metadata = json.load(f)

resnet18_cfg = DotDict()
for k, v in resnet18_run_metadata["params"].items():
    setattr(resnet18_cfg, k, v)
setattr(resnet18_cfg, "infer", True)

CB_MODELS_INPUT_PATH = CB_ARTIFACTS_INPUT_PATH / cb_cfg.models_output_dir
LGB_V1_MODELS_INPUT_PATH = LGB_V1_ARTIFACTS_INPUT_PATH / lgb_v1_cfg.models_output_dir
LGB_V2_MODELS_INPUT_PATH = LGB_V2_ARTIFACTS_INPUT_PATH / lgb_v2_cfg.models_output_dir
RESNET18_MODELS_INPUT_PATH = RESNET18_ARTIFACTS_INPUT_PATH / resnet18_cfg.models_output_dir

train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False)
test_metadata = pd.read_csv(INPUT_PATH / "test-metadata.csv", low_memory=False)
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

test_images = h5py.File(INPUT_PATH / "test-image.hdf5", mode="r")

folds_df = get_folds()
train_metadata = train_metadata.merge(folds_df, on=["isic_id", "patient_id"], how="inner")

id_column = "isic_id"
target_column = "target"
folds = train_metadata["fold"]

Train data size: (401059, 55)
Test data size: (3, 44)


In [3]:
def test_augment(image_size):
    transform = A.Compose([
        A.Resize(image_size, image_size),
#         A.Normalize(
#             mean=[0., 0., 0.],
#             std=[1, 1, 1],
#             max_pixel_value=255.0,
#             p=1.0
#         ),
        ToTensorV2()
    ], p=1.)
    return transform

class ISICDataset(Dataset):
    def __init__(self, metadata, images, augment, infer=False):
        self.metadata = metadata
        self.images = images
        self.augment = augment
        self.length = len(self.metadata)
        self.infer = infer
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        data = self.metadata.iloc[index]
        
        image = np.array(Image.open(BytesIO(self.images[data[id_column]][()])))
        image = self.augment(image=image)["image"]
        
        record = {
            "image": image
        }
        
        if not self.infer:
            target = data[target_column]
            record["target"] = torch.tensor(target).float()
        
        return record

class ISICNet(nn.Module):
    def __init__(self, arch="resnet18", pretrained=False, infer=False):
        super(ISICNet, self).__init__()
        self.infer = infer
        self.model = create_model(model_name=arch, pretrained=pretrained, in_chans=3,  num_classes=0, global_pool='')
        self.classifier = nn.Linear(self.model.num_features, 1)
        
        self.dropouts = nn.ModuleList([nn.Dropout(0.5) for i in range(5)])
        
    def forward(self, batch):
        image = batch["image"]
        image = image.float() / 255
        
        x = self.model(image)
        bs = len(image)
        pool = F.adaptive_avg_pool2d(x, 1).reshape(bs,-1)
        
        if self.training:
            logit = 0
            for i in range(len(self.dropouts)):
                logit += self.classifier(self.dropouts[i](pool))
            logit = logit/len(self.dropouts)
        else:
            logit = self.classifier(pool)
        return logit
    

all_folds = np.sort(folds.unique())
resnet18_test_predictions_df = pd.DataFrame({id_column: test_metadata[id_column]})
test_dataset = ISICDataset(test_metadata, test_images, augment=test_augment(image_size=resnet18_cfg.image_size), infer=True)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=resnet18_cfg.val_batch_size, num_workers=resnet18_cfg.val_num_worker, drop_last=False, pin_memory=True)
for fold in all_folds:
    accelerator = Accelerator(cpu=resnet18_cfg.cpu, mixed_precision=resnet18_cfg.mixed_precision)
    
    net = ISICNet(pretrained=False, infer=True)
    net = net.to(accelerator.device)
    
    net, test_dataloader = accelerator.prepare(net, test_dataloader)
    
    accelerator.load_state(RESNET18_MODELS_INPUT_PATH / 
                           f"fold_{fold}/model_{resnet18_cfg.model_name}_epoch_{resnet18_run_metadata['best_num_epochs'][f'fold_{fold}']}")
    
    net.eval()
    test_preds = []
    for step, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        # We could avoid this line since we set the accelerator with `device_placement=True`.
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}
        
        image0 = batch['image'].clone().detach()
        test_preds_batch = 0
        counter = 0
        with torch.no_grad():
            outputs = net(batch)
        preds = torch.sigmoid(outputs)
        preds = accelerator.gather_for_metrics((preds))
        test_preds_batch += preds.data.cpu().numpy().reshape(-1)
        counter += 1
        if resnet18_cfg.tta:
            batch["image"] = torch.flip(image0,dims=[2])
            with torch.no_grad():
                outputs = net(batch)
            preds = torch.sigmoid(outputs)
            preds = accelerator.gather_for_metrics((preds))
            test_preds_batch += preds.data.cpu().numpy().reshape(-1)
            counter += 1

            batch["image"] = torch.flip(image0,dims=[3])
            with torch.no_grad():
                outputs = net(batch)
            preds = torch.sigmoid(outputs)
            preds = accelerator.gather_for_metrics((preds))
            test_preds_batch += preds.data.cpu().numpy().reshape(-1)
            counter += 1

            for k in [1, 2, 3]:
                batch["image"] = torch.rot90(image0,k, dims=[2, 3])
                with torch.no_grad():
                    outputs = net(batch)
                preds = torch.sigmoid(outputs)
                preds = accelerator.gather_for_metrics((preds))
                test_preds_batch += preds.data.cpu().numpy().reshape(-1)
                counter += 1
                
        test_preds_batch = test_preds_batch / counter   
        test_preds.append(test_preds_batch)

    resnet18_test_predictions_df[f"fold_{fold}"] = np.concatenate(test_preds)

100%|██████████| 1/1 [00:00<00:00,  9.01it/s]


In [4]:
def stat_features(df, group_cols, value_col, stats):
    tmp = df.groupby(group_cols)[value_col].agg(stats)
    tmp.columns = [f"{value_col}_{stat}" for stat in stats]
    tmp.reset_index(inplace=True)
    df = df.merge(tmp, on=group_cols, how="left")
    df[f"{value_col}_mean_diff"] = df[value_col] - df[f"{value_col}_mean"]
    return df


def feature_engineering(df):
    # v1 features
    df["lesion_size_ratio"]              = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
    df["hue_contrast"]                   = (df["tbp_lv_H"] - df["tbp_lv_Hext"]).abs()
    df["normalized_lesion_size"]         = df["clin_size_long_diam_mm"] / df["age_approx"]
    df["overall_color_difference"]       = (df["tbp_lv_deltaA"] + df["tbp_lv_deltaB"] + df["tbp_lv_deltaL"]) / 3
    patient_num_images = df.groupby("patient_id", as_index=False)["isic_id"].count().rename(columns={"isic_id": "num_images"})
    df = df.merge(patient_num_images, on="patient_id", how="left")
    
    # v2 features
    df["max_minus_min_lesion_size"] = df["clin_size_long_diam_mm"] - df["tbp_lv_minorAxisMM"]
    
    df["tbp_lv_A_diff"] =  df["tbp_lv_Aext"] - df["tbp_lv_A"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_A_diff", ["mean"])
    
    df["tbp_lv_B_diff"] =  df["tbp_lv_Bext"] - df["tbp_lv_B"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_B_diff", ["mean"])
    
    df["tbp_lv_L_diff"] =  df["tbp_lv_Lext"] - df["tbp_lv_L"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_L_diff", ["mean"])
    
    df["tbp_lv_L_std_diff"] =  df["tbp_lv_stdLExt"] - df["tbp_lv_stdL"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_L_std_diff", ["mean"])
    
    return df

train_metadata = feature_engineering(train_metadata.copy())
test_metadata = feature_engineering(test_metadata.copy())

In [5]:
with open(CB_ARTIFACTS_INPUT_PATH / f"{cb_cfg.model_name}_encoder.joblib", "rb") as f:
    mixed_encoded_preprocessor = joblib.load(f)

enc = mixed_encoded_preprocessor.fit(train_metadata)
X_test = enc.transform(test_metadata)

columns_for_model = len(X_test.columns)
print(f"Total number of columns: {columns_for_model}")

all_folds = np.sort(folds.unique())
cb_test_predictions_df = pd.DataFrame({id_column: test_metadata[id_column]})
for fold in all_folds:
    model = cb.CatBoostClassifier(use_best_model=True)
    model.load_model(CB_MODELS_INPUT_PATH / f"{cb_cfg.model_name}_fold_{fold}.txt")
    cb_test_predictions_df[f"fold_{fold}"] = model.predict_proba(X_test)[:, -1]

Total number of columns: 46


In [6]:
with open(LGB_V1_ARTIFACTS_INPUT_PATH / f"{lgb_v1_cfg.model_name}_encoder.joblib", "rb") as f:
    mixed_encoded_preprocessor = joblib.load(f)

enc = mixed_encoded_preprocessor.fit(train_metadata)
X_test = enc.transform(test_metadata)

columns_for_model = len(X_test.columns)
print(f"{lgb_v1_cfg.model_name}: Total number of columns: {columns_for_model}")

all_folds = np.sort(folds.unique())
lgb_v1_test_predictions_df = pd.DataFrame({id_column: test_metadata[id_column]})
for fold in all_folds:
    model = lgb.Booster(model_file=LGB_V1_MODELS_INPUT_PATH / f"{lgb_v1_cfg.model_name}_fold_{fold}.txt")
    lgb_v1_test_predictions_df[f"fold_{fold}"] = model.predict(X_test, num_iteration=lgb_v1_run_metadata["best_num_rounds"][f"fold_{fold}"])

lgb_v1: Total number of columns: 46


In [7]:
with open(LGB_V2_ARTIFACTS_INPUT_PATH / f"{lgb_v2_cfg.model_name}_encoder.joblib", "rb") as f:
    mixed_encoded_preprocessor = joblib.load(f)

enc = mixed_encoded_preprocessor.fit(train_metadata)
X_test = enc.transform(test_metadata)

columns_for_model = len(X_test.columns)
print(f"{lgb_v2_cfg.model_name}: Total number of columns: {columns_for_model}")

all_folds = np.sort(folds.unique())
lgb_v2_test_predictions_df = pd.DataFrame({id_column: test_metadata[id_column]})
for fold in all_folds:
    model = lgb.Booster(model_file=LGB_V2_MODELS_INPUT_PATH / f"{lgb_v2_cfg.model_name}_fold_{fold}.txt")
    lgb_v2_test_predictions_df[f"fold_{fold}"] = model.predict(X_test, num_iteration=lgb_v2_run_metadata["best_num_rounds"][f"fold_{fold}"])

lgb_v2: Total number of columns: 20


In [8]:
cb_test_predictions_df[target_column] = cb_test_predictions_df[[f"fold_{fold}" for fold in all_folds]].mean(axis=1)
lgb_v1_test_predictions_df[target_column] = lgb_v1_test_predictions_df[[f"fold_{fold}" for fold in all_folds]].mean(axis=1)
lgb_v2_test_predictions_df[target_column] = lgb_v2_test_predictions_df[[f"fold_{fold}" for fold in all_folds]].mean(axis=1)
resnet18_test_predictions_df[target_column] = resnet18_test_predictions_df[[f"fold_{fold}" for fold in all_folds]].mean(axis=1)

In [9]:
test_predictions_df = (
    cb_test_predictions_df[[id_column, target_column]].merge(
        lgb_v1_test_predictions_df[[id_column, target_column]], on=id_column, suffixes=("_cb", "")).merge(
            lgb_v2_test_predictions_df[[id_column, target_column]], on=id_column, suffixes=("_lgb_v1", "")).merge(
                resnet18_test_predictions_df[[id_column, target_column]], on=id_column, suffixes=("_lgb_v2", "_resnet18"))
    )

In [10]:
test_predictions_df

,isic_id,target_cb,target_lgb_v1,target_lgb_v2,target_resnet18
0,ISIC_0015657,0.000180,0.000226,0.000296,0.023122
1,ISIC_0015729,0.000033,0.000206,0.000322,0.004345
2,ISIC_0015740,0.000514,0.000303,0.000332,0.001377


In [11]:
test_predictions_df["target_cb"] = test_predictions_df["target_cb"].rank(pct=True).values
test_predictions_df["target_lgb_v1"] = test_predictions_df["target_lgb_v1"].rank(pct=True).values
test_predictions_df["target_lgb_v2"] = test_predictions_df["target_lgb_v2"].rank(pct=True).values
test_predictions_df["target_resnet18"] = test_predictions_df["target_resnet18"].rank(pct=True).values

In [12]:
test_predictions_df[target_column] = (test_predictions_df["target_cb"] * 0.26 + 
                                      test_predictions_df["target_lgb_v1"] * 0.26 + 
                                      test_predictions_df["target_lgb_v1"] * 0.22 + 
                                      test_predictions_df["target_resnet18"] * 0.26)

In [13]:
test_predictions_df.head()

,isic_id,target_cb,target_lgb_v1,target_lgb_v2,target_resnet18,target
0,ISIC_0015657,0.666667,0.666667,0.333333,1.000000,0.753333
1,ISIC_0015729,0.333333,0.333333,0.666667,0.666667,0.420000
2,ISIC_0015740,1.000000,1.000000,1.000000,0.333333,0.826667


In [14]:
test_predictions_df[target_column].describe()

count    3.000000
mean     0.666667
std      0.216744
min      0.420000
25%      0.586667
50%      0.753333
75%      0.790000
max      0.826667
Name: target, dtype: float64

In [15]:
test_predictions_df[[id_column, target_column]].head()

,isic_id,target
0,ISIC_0015657,0.753333
1,ISIC_0015729,0.420000
2,ISIC_0015740,0.826667


In [16]:
test_predictions_df[[id_column, target_column]].to_csv("submission.csv", index=False)